In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd
from tqdm import tqdm
import metnum
import numpy
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score,precision_score,recall_score
import csv
import time


In [3]:
def dividir_en_training_testing(M,percentage):
    limit = int(percentage * M.shape[0]) 
    M_train = M[:limit]
    M_val = M[limit:]
    return M_train,M_val


In [4]:
def dividir_en_X_y(M):
    X = M[M.columns[1:]].values
    y = M["label"].values.reshape(-1, 1)
    return X,y

In [5]:
def cargar_datos_de_experimento(percentage,cant_muestra=42000):
    df_train = pd.read_csv("../data/train.csv")
    df_train = shuffle(df_train)
    df_train = df_train[:cant_muestra]
    X,y = dividir_en_X_y(df_train)
    X_train,X_val = dividir_en_training_testing(X,percentage)
    y_train,y_val = dividir_en_training_testing(y,percentage)
    return X_train,X_val,y_train,y_val
    

In [10]:
def correr_Knn_con_todos_los_k_posibles(porcentage_para_entrenar,cant_muestras=42000,semilla = 2 ):
    df_train = pd.read_csv("../data/train.csv")
    df_train =  df_train.sample(frac=1, random_state=semilla)
    X= df_train[df_train.columns[1:]].values
    y = df_train["label"].values.reshape(-1, 1)

    accuracy_of_all_k = []
    precision_df = pd.DataFrame(columns=(0,1,2,3,4,5,6,7,8,9))
    recall_df = pd.DataFrame(columns=(0,1,2,3,4,5,6,7,8,9))
    times = [] 
        
    setup_time_start_time = time.time()
    
    limit = int(0.8 * X.shape[0]) 
    X_train = X[:limit]
    X_val = X[limit:]
    y_train = y[:limit]
    y_val = y[limit:]
    alpha = 0
    # Hago el fit generico que guarda los datos 

    setup_time_end_time = time.time()
    setup_time = setup_time_start_time-setup_time_end_time
    for k in tqdm(range(1,2000,100)):

        # Correr knn 
        clf = metnum.KNNClassifier(k)
        knn_start_time = time.time()
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_val)
        print(accuracy_score(y_val, y_pred))
        knn_end_time = time.time()
        knn_time = knn_end_time-knn_start_time
        # Calcular metricas de interes
        labels= [0,1,2,3,4,5,6,7,8,9]
        precision = precision_score(y_val, y_pred,labels=labels, average=None)
        accuracy =  accuracy_score(y_val, y_pred)
        recall   =  recall_score(y_val,y_pred,labels=labels, average=None)

        # Escribir los resultados
        accuracy_of_all_k.append([alpha, k ,accuracy])
        times.append(setup_time+knn_time)

            # Agregar una fila al dataframe de precision
        digit = 0 
        precision_dict={}
        for i in range(0,10,1):
            precision_dict[digit]=precision[i]
            digit += 1 
        precision_dict['k']=k
        precision_dict['alpha']=alpha
        precision_df = precision_df.append(precision_dict,ignore_index=True)

             # Agregar una fila al dataframe de recall
        digit = 0 
        recall_dict={}
        for i in range(0,10,1):
            recall_dict[digit]=recall[i]
            digit += 1 
        recall_dict['k']=k
        recall_dict['alpha']=alpha
        recall_df = recall_df.append(recall_dict,ignore_index=True)

    # Escribo los resultados a un archivo para no tener que correr devuelta los resultados.
    precision_df.to_csv('knn_solo_precision.csv', index=False) 
    recall_df.to_csv('knn_solo_recall.csv', index=False)   

    with open('knn_solo_acuracy.csv', 'w', newline='') as myfile:
        wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
        wr.writerow(accuracy_of_all_k)

    with open('knn_solo_time.csv', 'w', newline='') as myfile:
        wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
        wr.writerow(times)


    

In [11]:
correr_Knn_con_todos_los_k_posibles(0.8)

  5%|▌         | 1/20 [09:12<2:55:03, 552.84s/it]

0.9673809523809523


 10%|█         | 2/20 [18:04<2:42:11, 540.65s/it]

0.9329761904761905


 15%|█▌        | 3/20 [26:27<2:28:14, 523.21s/it]

...

0.7867857142857143


 95%|█████████▌| 19/20 [2:41:31<07:17, 437.85s/it]

0.7813095238095238


100%|██████████| 20/20 [2:48:36<00:00, 505.81s/it]

0.7770238095238096


In [2]:
def correr_Knn_pca(porcentage_para_entrenar,cant_muestras=42000,semilla = 2):
    df_train = pd.read_csv("../data/train.csv")
    df_train =  df_train.sample(frac=1, random_state=semilla)
    X_original = df_train[df_train.columns[1:]].values
    y = df_train["label"].values.reshape(-1, 1)

    
    accuracy_of_all_k = []
    precision_df = pd.DataFrame(columns=('alpha','k',0,1,2,3,4,5,6,7,8,9))
    recall_df = pd.DataFrame(columns=('alpha','k',0,1,2,3,4,5,6,7,8,9))
    times = [] 
    for alpha in tqdm(range(1,29,4)):
        
        setup_time_start_time = time.time()
        pca = metnum.PCA(alpha)
        X = pca.transform(X_original)
        limit = int(0.8 * X.shape[0]) 
        X_train = X[:limit]
        X_val = X[limit:]
        y_train = y[:limit]
        y_val = y[limit:]

        
        # Hago el fit generico que guarda los datos 

        setup_time_end_time = time.time()
        setup_time = setup_time_start_time-setup_time_end_time
        
        for k in tqdm(range(1,1000,50)):

            # Correr knn 
            clf = metnum.KNNClassifier(k)
  
            knn_start_time = time.time()
            clf.fit(X_train, y_train)
            y_pred = clf.predict(X_val)
            print(accuracy_score(y_val, y_pred))
            knn_end_time = time.time()
            knn_time = knn_end_time-knn_start_time
            # Calcular metricas de interes
            labels= [0,1,2,3,4,5,6,7,8,9]
            precision = precision_score(y_val, y_pred,labels=labels, average=None)
            accuracy =  accuracy_score(y_val, y_pred)
            recall   =  recall_score(y_val,y_pred,labels=labels, average=None)

            # Escribir los resultados
            accuracy_of_all_k.append([alpha, k ,accuracy])
            times.append(setup_time+knn_time)

                # Agregar una fila al dataframe de precision
            digit = 0 
            precision_dict={}
            for i in range(0,10,1):
                precision_dict[digit]=precision[i]
                digit += 1 
            precision_dict['k']=k
            precision_dict['alpha']=alpha
            precision_df = precision_df.append(precision_dict,ignore_index=True)

                 # Agregar una fila al dataframe de recall
            digit = 0 
            recall_dict={}
            for i in range(0,10,1):
                recall_dict[digit]=recall[i]
                digit += 1 
            recall_dict['k']=k
            recall_dict['alpha']=alpha
            recall_df = recall_df.append(recall_dict,ignore_index=True)

        # Escribo los resultados a un archivo para no tener que correr devuelta los resultados.
        precision_df.to_csv('knn_pca_precision.csv', index=False) 
        recall_df.to_csv('knn_pca_recall.csv', index=False)   

        with open('knn_pca_acuracy.csv', 'w', newline='') as myfile:
            wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
            wr.writerow(accuracy_of_all_k)

        with open('knn_pca_time.csv', 'w', newline='') as myfile:
            wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
            wr.writerow(times)

 

    

In [3]:
correr_Knn_pca(0.8)

  5%|▌         | 1/20 [00:40<12:56, 40.88s/it]

0.2536904761904762



 10%|█         | 2/20 [01:18<11:38, 38.81s/it]

0.2922619047619048



 15%|█▌        | 3/20 [01:55<10:50, 38.25s/it]

...

0.8863095238095238



 95%|█████████▌| 19/20 [16:32<00:52, 52.39s/it]

0.8832142857142857



100%|██████████| 7/7 [1:52:17<00:00, 962.46s/it] 

0.8803571428571428


In [3]:
%load_ext autoreload
%autoreload 2
import pandas as pd

df_train = pd.read_csv("../data/train.csv")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
# Desordeno los datos
df_train = df_train.sample(frac=1, random_state=2)

# Separo los datos en pixeles y etiquetas y los guardo en arrays de numpy
X = df_train[df_train.columns[1:]].values
y = df_train["label"].values.reshape(-1, 1)

# Aplico PCA
pca = metnum.PCA(15)
X = pca.transform(X)

# Aplico Knn
clf = metnum.KNNClassifier(1000)
acc = []

# Divido en train testing
limit = int(0.8 * X.shape[0]) 
X_train = X[:limit]
X_val = X[limit:]
y_train = y[:limit]
y_val = y[limit:]

clf.fit(X_train, y_train)
y_pred = clf.predict(X_val)
print(accuracy_score(y_val, y_pred))

0.8683333333333333
